Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

#################
# My notebook issues
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
sys.path.append('home/egupta/.local/bin')

##############

import os
import numpy as np
import scipy.stats as ss
import networkx as nx
import csv 
import numpy as np
import seaborn as sns
from pandas import DataFrame
from scipy.optimize import curve_fit
import matplotlib
from matplotlib import pyplot as plt

%matplotlib inline

##qtensor imports
import qtensor
from qtensor import QAOA_energy
from qtensor import parameter_optimization as popt
import torch

2021-07-23 09:16:44,848- WARNING•	QuickBB solver is unavailable
2021-07-23 09:16:44,851- WARNING•	Tamaki solver is unavailable: No path /home/egupta/.local/lib/python3.7/site-packages/thirdparty/tamaki_treewidth


# Grabbing Graphs

In [2]:
resi = []
with open('20NodeRandomGraph1 (2).csv', 'r', newline = '') as g:
  spamreader = csv.reader(g, quotechar='|')
  for row in spamreader:
    resi.append(row)

#convert bunch of rows to matrices
n = int(len(resi)/20)
mat_list = []
for i in range(n):
  array = []
  for j in range(20):
    row = resi[i*20 +j]
    row = [float(k) for k in row]
    array.append(row)
  mat_list.append(array)

In [3]:
def mats_to_graphs(mat_list):
  """
  Given list of adjaceny matrix repr of graphs, converts to actual graphs
  """
  g_list = []
  for mat in mat_list:
    arr = np.array(mat)
    G = nx.convert_matrix.from_numpy_matrix(arr)
    g_list.append(G)
  return g_list

In [4]:
g_list = mats_to_graphs(mat_list)

In [5]:
len(g_list)

55

In [6]:
new_g_list = g_list[28:]

In [7]:
len(new_g_list)

27

# FindingLocal Optima

Old Function

In [8]:

def get_qaoa_params(graph, edge=None, gamma=[0.5], beta=[0.5], steps=50):
    loss_history, params_history = popt.qaoa_maxcut_torch(graph, gamma, beta,
                                                          ordering_algo='greedy',
                                                          Opt=torch.optim.RMSprop,
                                                          edge=edge,
                                                          opt_kwargs=dict(lr=0.002),
                                                          steps=steps,
                                                          pbar=False)
    best_ix = np.argmin(loss_history)
    return params_history[best_ix], loss_history[best_ix]

def get_edge_contribution(graph, edge, gamma, beta):
    sim = qtensor.QtreeSimulator()
    composer = qtensor.DefaultQAOAComposer(graph, gamma=gamma, beta=beta)
    composer.energy_expectation_lightcone(edge)
    return np.real(sim.simulate_batch(composer.circuit)[0])

def opt_runs(G, seeds = 20, steps = 200):
  """
  Doing multiple optimization runs, each with random initialized param
  Returning results which inlclude the optimal param and corresponding energies
  """

  #first do 20 optimization runs 
  s = 0
  g_results = []
    
  while s<seeds:
    print('seed #' + str(s))
    #initialize param
    gamma = [np.random.rand()]
    beta = [np.random.rand() ]
    initial_energy = QAOA_energy(G,gamma =  gamma, beta = beta)

    #optimization
    params, loss = get_qaoa_params(G, gamma=gamma, beta=beta,edge = None, steps=steps)
    #print('Finished get_qaoa_params')
    gamma_opt, beta_opt = params[0], params[1]
    opt_energy = QAOA_energy(G, gamma = gamma_opt, beta = beta_opt)

    result = np.asarray([float(gamma_opt), float(beta_opt), float(opt_energy), float(initial_energy)])
    g_results.append(result)

    # comparing with max
    s+=1

  
  return g_results

Execution Code

In [ ]:
results = []
graph = 0
for g in new_g_list:
    graph+=1
    print(graph)
    result = opt_runs(G=g, seeds =20, steps = 200)
    results.append(result)

1
seed #0



seed #1



seed #2



seed #3



seed #4


...


seed #13



seed #14



seed #15



seed #16



seed #17


In [13]:
results

[[array([ 0.08274417,  0.38094306, 23.33865618, 16.28225796]),
  array([ 0.5737955 ,  0.38407898, 19.5901115 , 17.9241676 ]),
  array([ 0.08274328,  0.88088864, 23.33865443, 19.28751811]),
  array([ 0.9172563 ,  0.61908907, 23.33865548, 14.75677741]),
  array([ 0.08206806,  0.38095185, 23.33832052, 20.09238194]),
  array([ 0.91724688,  0.11904871, 23.33865615, 18.18372242]),
  array([ 0.91721517,  0.61904788, 23.338655  , 17.69924273]),
  array([ 0.0827514 ,  0.38095126, 23.33865617, 18.86484638]),
  array([ 0.42623669,  0.6159268 , 19.5901118 , 18.7789423 ]),
  array([ 0.42623642,  0.61592537, 19.5901118 , 18.26774182]),
  array([ 0.42623571,  0.61591816, 19.59011179, 17.74869234]),
  array([ 0.42623362,  0.61588383, 19.59011153, 18.52074015]),
  array([ 0.91724235,  0.11904851, 23.33865608, 17.22005131]),
  array([ 0.08274354,  0.88091487, 23.33865561, 17.38650806]),
  array([ 0.91725487,  1.11899209, 23.33865474, 15.28573566]),
  array([ 0.5737583 ,  0.38407111, 19.5901118 , 18.6585

In [15]:
len(results[0])

20

# Storing Data

In [16]:
with open('55_20NodeRandom_Local_Data_Part2.csv', 'w', newline = '') as f:
  thewriter = csv.writer(f)
  for g_results in results :
        for seed_results in g_results:
            thewriter.writerow(seed_results)